In [89]:
# Core & Gmail imports
import base64
import pickle
import os
from email.mime.text import MIMEText

# Google Gmail API
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# LLM + UI
from openai import OpenAI


In [90]:
SCOPES = ["https://www.googleapis.com/auth/gmail.send"]


In [91]:
def send_gmail(to_email, subject, body):
    creds = None

    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                os.path.join(os.getcwd(), "credentials.json"),
                SCOPES
            )
            creds = flow.run_local_server(port=0)

        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    service = build("gmail", "v1", credentials=creds)

    # Send as HTML so spacing looks correct
    message = MIMEText(body.replace("\n", "<br>"), "html")
    message["to"] = to_email
    message["subject"] = subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    service.users().messages().send(
        userId="me", body={"raw": raw}
    ).execute()


In [92]:
openai = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)


In [93]:
SYSTEM_PROMPT = """
You are an intelligent email-writing assistant.

STRICT RULES (must follow exactly):
- Always output in Markdown
- Always include these exact headings:
  ## Subject
  ## Email Body
- Do NOT change the heading text
- Do NOT omit any section
"""


In [94]:
def generate_email(context, key_points, name, dates):
    user_prompt = f"""
Write a professional email using the details below.

Context:
{context}

Dates involved:
{dates}

Key points:
{key_points}

Sender name:
{name}

Naturally include the dates in the email where appropriate.
"""
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content


In [95]:
def parse_email(md_text):
    subject = "No Subject"
    body = md_text.strip()

    if "## Subject" in md_text:
        part = md_text.split("## Subject", 1)[1]
        if "## Email Body" in part:
            subject = part.split("## Email Body", 1)[0].strip()
            body = part.split("## Email Body", 1)[1].strip()

    return subject, body


In [96]:
LAST_GENERATED_EMAIL = None


In [97]:
def preview_email(context, key_points, name, dates):
    global LAST_GENERATED_EMAIL
    LAST_GENERATED_EMAIL = generate_email(context, key_points, name, dates)
    return LAST_GENERATED_EMAIL


In [98]:
def send_previewed_email(to_email):
    global LAST_GENERATED_EMAIL

    if LAST_GENERATED_EMAIL is None:
        return "❌ Please generate an email first."

    subject, body = parse_email(LAST_GENERATED_EMAIL)
    send_gmail(to_email, subject, body)

    return "✅ Email sent successfully!"


In [106]:
import gradio as gr

with gr.Blocks(title="AI Email Generator") as demo:

    gr.Markdown("## ✉️ AI Email Generator")

    context = gr.Textbox(
        label="What is the email about?",
        placeholder="e.g., Academic lab absence"
    )

    dates = gr.Textbox(
        label="Relevant Dates",
        placeholder="e.g., Thursday, 12 Sept 2025"
    )

    key_points = gr.Textbox(
        label="Key Points",
        lines=5,
        placeholder="- Reason\n- Request\n- Submission plan"
    )

    name = gr.Textbox(
        label="Your Name",
        placeholder="e.g., Sudhanshu Batra"
    )

    to_email = gr.Textbox(
        label="Send To (Email)"
    )

    generate_btn = gr.Button("🔄 Generate / Preview")
    send_btn = gr.Button("📨 Send Email")

    preview = gr.Markdown()
    status = gr.Markdown()

    generate_btn.click(
        fn=preview_email,
        inputs=[context, key_points, name, dates],
        outputs=preview
    )

    send_btn.click(
        fn=send_previewed_email,
        inputs=to_email,
        outputs=status
    )

demo.launch(inbrowser=True,share=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://777041da2e6224992b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
